## Deliverable 2. Create a Customer Travel Destinations Map.

In [3]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [5]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Cape Town,ZA,-33.9258,18.4232,74.88,60,0,1.01,clear sky
1,1,Tuy Hoa,VN,13.0833,109.3000,71.78,88,100,12.41,overcast clouds
2,2,Pemangkat,ID,1.1667,108.9667,78.17,87,100,7.76,overcast clouds
3,3,Skibbereen,IE,51.5500,-9.2667,52.11,84,100,31.14,overcast clouds
4,4,Grahamstown,ZA,-33.3042,26.5328,62.47,89,100,20.87,overcast clouds


In [7]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 55
What is the maximum temperature you would like for your trip? 75


In [8]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Cape Town,ZA,-33.9258,18.4232,74.88,60,0,1.01,clear sky
1,1,Tuy Hoa,VN,13.0833,109.3000,71.78,88,100,12.41,overcast clouds
4,4,Grahamstown,ZA,-33.3042,26.5328,62.47,89,100,20.87,overcast clouds
5,5,Flinders,AU,-34.5833,150.8552,68.38,81,99,5.99,moderate rain
8,8,Busselton,AU,-33.6500,115.3333,65.05,83,0,7.56,clear sky
11,11,Punta Arenas,CL,-53.1500,-70.9167,59.11,77,100,21.85,light intensity shower rain
12,12,Cidreira,BR,-30.1811,-50.2056,73.99,94,100,8.93,overcast clouds
14,14,Padang,ID,-0.9492,100.3543,74.16,97,100,1.70,overcast clouds
15,15,Saint George,US,37.1041,-113.5841,56.35,26,0,3.00,clear sky
16,16,Esperance,AU,-33.8667,121.9000,66.04,70,5,17.47,clear sky


In [9]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                202
City                   202
Country                198
Lat                    202
Lng                    202
Max Temp               202
Humidity               202
Cloudiness             202
Wind Speed             202
Current Description    202
dtype: int64

In [11]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
print(clean_df.count())

City_ID                198
City                   198
Country                198
Lat                    198
Lng                    198
Max Temp               198
Humidity               198
Cloudiness             198
Wind Speed             198
Current Description    198
dtype: int64


In [12]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Cape Town,ZA,74.88,clear sky,-33.9258,18.4232,
1,Tuy Hoa,VN,71.78,overcast clouds,13.0833,109.3000,
4,Grahamstown,ZA,62.47,overcast clouds,-33.3042,26.5328,
5,Flinders,AU,68.38,moderate rain,-34.5833,150.8552,
8,Busselton,AU,65.05,clear sky,-33.6500,115.3333,
11,Punta Arenas,CL,59.11,light intensity shower rain,-53.1500,-70.9167,
12,Cidreira,BR,73.99,overcast clouds,-30.1811,-50.2056,
14,Padang,ID,74.16,overcast clouds,-0.9492,100.3543,
15,Saint George,US,56.35,clear sky,37.1041,-113.5841,
16,Esperance,AU,66.04,clear sky,-33.8667,121.9000,


In [13]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [22]:
# 7. Drop the rows where there is no Hotel Name.
print(len(hotel_df))
print(hotel_df.count())
clean_hotel_df = hotel_df.loc[hotel_df["Hotel Name"] != ""]
print(len(clean_hotel_df))
print(clean_hotel_df.count())
clean_hotel_df.head(10)

198
City                   198
Country                198
Max Temp               198
Current Description    198
Lat                    198
Lng                    198
Hotel Name             198
dtype: int64
184
City                   184
Country                184
Max Temp               184
Current Description    184
Lat                    184
Lng                    184
Hotel Name             184
dtype: int64


,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Cape Town,ZA,74.88,clear sky,-33.9258,18.4232,Southern Sun Waterfront Cape Town
1,Tuy Hoa,VN,71.78,overcast clouds,13.0833,109.3000,CenDeluxe Hotel Tuy Hoa
4,Grahamstown,ZA,62.47,overcast clouds,-33.3042,26.5328,Bartholomew's Loft
5,Flinders,AU,68.38,moderate rain,-34.5833,150.8552,Shellharbour Resort & Conference Centre
8,Busselton,AU,65.05,clear sky,-33.6500,115.3333,Observatory Guest House
11,Punta Arenas,CL,59.11,light intensity shower rain,-53.1500,-70.9167,Hotel Hain
12,Cidreira,BR,73.99,overcast clouds,-30.1811,-50.2056,Hotel Castelo
14,Padang,ID,74.16,overcast clouds,-0.9492,100.3543,Mercure Padang
15,Saint George,US,56.35,clear sky,37.1041,-113.5841,Best Western Plus Abbey Inn
16,Esperance,AU,66.04,clear sky,-33.8667,121.9000,Hospitality Esperance


In [23]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [24]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country Code</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [25]:
# 11a. Add a marker layer for each city to the map. 
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig


Figure(layout=FigureLayout(height='420px'))